## Analyse Resoluties in het Rampjaar

Vragen:
- rondom commissies?
- administrative veranderingen
- wat kunnen we vinden in de ordinaris resoluties?
- vertegenwoordiging van gewesten

- 25 februari
- April: aanleiding begin oorlog
- zeeslagen in juni
- moord op de Witt broers

- sleutelmomenten bij stichting rampjaar


In [8]:
# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/femkegordijn/republic-project


In [6]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es(host_type='external', timeout=60)

In [5]:
!conda install elasticsearch

failed

CondaError: KeyboardInterrupt



In [7]:
# use single word or multi-word phrase
for hit in rep_es.keyword_in_context("voornoemde Procureur"):
    print(hit["context"])

ApiError: ApiError(406, 'Content-Type header [application/vnd.elasticsearch+json; compatible-with=8] is not supported', 'Content-Type header [application/vnd.elasticsearch+json; compatible-with=8] is not supported')

In [9]:
import gzip
import json

import republic.model.republic_document_model as rdm


resolutions_file = "../../data/resolutions/rampjaar-ordinaris-resolutions.json.gz"

# Reading the JSON representations from file again and turning 
# them into Resolution objects again
with gzip.open(resolutions_file, 'rt') as fh:
    # the document model has a convenience function to turn a JSON representation
    # to a Resolution object: json_to_republic_resolution
    resolutions_1672 = [rdm.json_to_republic_resolution(res) for res in json.load(fh)]
    

FileNotFoundError: [Errno 2] No such file or directory: '../../data/resolutions/rampjaar-ordinaris-resolutions.json.gz'

In [5]:
import gzip

para_file = "../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz"

with gzip.open(para_file, 'wt') as fh:
    for res in resolutions_1672:
        for para in res.paragraphs:
            fh.write(f"{res.id}\t{para.id}\t{para.text}\n")

In [3]:
from gensim.models import Word2Vec
import republic.helper.text_helper as text_helper

para_file = "../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz"

res_sents = text_helper.ResolutionSentences(para_file)

for sent in res_sents:
    print(sent)
    break

parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
['jovis', 'den', '7']


In [19]:
model_rampjaar = Word2Vec(sentences=res_sents, vector_size=100, window=100, min_count=3, workers=4)

parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz


In [20]:
model_rampjaar.wv.most_similar('periculeuse')

[('secreete', 0.8964669108390808),
 ('kondtschappen', 0.8894565105438232),
 ('crijsch', 0.8885893225669861),
 ('nadruck', 0.8840699195861816),
 ('ticheden', 0.8839587569236755),
 ('onser', 0.8797962069511414),
 ('voornamentlick', 0.8755388855934143),
 ('beswooren', 0.8736085295677185),
 ('complete', 0.8719626069068909),
 ('ritmeesteren', 0.8708713054656982)]

In [14]:
para_files = [
    "../../data/paragraphs/resolutions-paragraphs-random.tsv.gz",
    "../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz"
]

res_sents = text_helper.ResolutionSentences(para_files)

model_rampjaar_big = Word2Vec(sentences=res_sents, vector_size=100, window=5, min_count=3, workers=4)

parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz


In [16]:
model_rampjaar_big = model_rampjaar

In [17]:
model_rampjaar_big.wv.most_similar('periculeuse')

[('kostelijke', 0.5713287591934204),
 ('fatigues', 0.5642806887626648),
 ('belegeringen', 0.5626934170722961),
 ('abdurahman', 0.5579507946968079),
 ('duwen', 0.5528756380081177),
 ('inductien', 0.5463021397590637),
 ('onvermijdelijcke', 0.5394834876060486),
 ('felle', 0.5383666157722473),
 ('gins', 0.5367465615272522),
 ('collusien', 0.5363394021987915)]

In [22]:
misspelling_dict_file = '../../ground_truth/correction/misspelled_common_terms-corrections.tsv'

rewrite = text_helper.read_rewrite_dictionary(misspelling_dict_file, include_uncertain=False)

res_sents = text_helper.ResolutionSentences(para_files, lowercase=True, to_ascii=True, rewrite_dict=rewrite)


In [23]:
model_rampjaar_bg_norm = Word2Vec(sentences=res_sents, vector_size=100, window=5, min_count=3, workers=4)

parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-random.tsv.gz
parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz


In [32]:
model_rampjaar_bg_norm.wv.most_similar('periculeuse', topn=25)

[('kostelijke', 0.6283071637153625),
 ('onvoorsichtigheyt', 0.6088605523109436),
 ('praghtige', 0.6071582436561584),
 ('overmacht', 0.6026226878166199),
 ('wintertijdt', 0.5964421033859253),
 ('zeden', 0.5911127328872681),
 ('voorsichticheijt', 0.5852442383766174),
 ('tormenten', 0.5846596360206604),
 ('geduren', 0.5791469812393188),
 ('renaitre', 0.5742583870887756),
 ('cordaetheyt', 0.5725923180580139),
 ('battailles', 0.572464644908905),
 ('quetsuren', 0.5723572969436646),
 ('invoeringh', 0.5713678002357483),
 ('tineert', 0.5713167190551758),
 ('veroorsaeckten', 0.5713120102882385),
 ('knien', 0.5711950659751892),
 ('dispendieuse', 0.5711163282394409),
 ('eyfsch', 0.5708966255187988),
 ('betert', 0.569885790348053),
 ('steng', 0.5691151022911072),
 ('abry', 0.5683889985084534),
 ('attacques', 0.5683482885360718),
 ('trouweloose', 0.5683440566062927),
 ('vigoureuss', 0.5679720044136047)]

In [26]:
model_rampjaar_bg_norm.wv.most_similar(positive=['periculeuse', 'tormenten'])

[('overmacht', 0.7309702038764954),
 ('moetwillige', 0.6839448809623718),
 ('attacques', 0.6816102266311646),
 ('kostelijke', 0.6810553669929504),
 ('quellingen', 0.6772211194038391),
 ('fatigues', 0.6722176671028137),
 ('gedurige', 0.6714029908180237),
 ('quetsuren', 0.665451169013977),
 ('mishandelinge', 0.6651085019111633),
 ('stantvastigheyt', 0.6617364287376404)]

In [33]:
for hit in rep_es.keyword_in_context("trouweloose", context_size=5):
    print(hit["context"])

                         haar Hoog Mog. , aangesien de trouweloose verbreeking van de Vreede met
                   schraapsugt aangedaan, en van sijne trouweloose desertie uit den dienst van
                        op eene soo onregtvaardige als trouweloose wyse aangevallen, en in een
               subtile vindingen ende bedenckingen van trouweloose Bediendens, bedriegeryen van Leveranciers en


In [48]:
query = {
    "bool": {
        "must": [
            #{"match": {"type": "resolution"}},
            {"match": {"metadata.session_year": 1672}}
            #{"match": {"metadata.proposition_type": "requeste"}}
        ]
    }
}
aggs = {
    "months": {
        "date_histogram": {
            "field": "metadata.session_date",
            "calendar_interval": "month"
        }
    }
}
response = rep_es.es_anno.search(index="resolutions", query=query, aggs=aggs, size=10000)
buckets = response["aggregations"]["months"]["buckets"]
sum_buckets = 0
for bucket in buckets:
    print(bucket["key_as_string"].split("T")[0], bucket["doc_count"])
    sum_buckets += bucket["doc_count"]
    
print("Total:", sum_buckets)

resolutions_1672 = [hit["_source"] for hit in response["hits"]["hits"]]
len(resolutions_1672)

1672-01-01 218
1672-02-01 207
1672-03-01 260
1672-04-01 116
1672-05-01 311
1672-06-01 191
1672-07-01 209
1672-08-01 226
1672-09-01 239
1672-10-01 203
1672-11-01 227
1672-12-01 198
Total: 2605


2605

In [50]:
from collections import Counter

Counter([res["metadata"]["proposition_type"] if "proposition_type" in res["metadata"] else "missing" for res in resolutions_1672])

Counter({'missing': 193,
         None: 460,
         'missive': 1617,
         'requeste': 289,
         'rekening': 1,
         'memorie': 36,
         'declaratie': 7,
         'rapport': 2})

In [5]:
query = {
    "bool": {
        "must": [
            #{"match": {"type": "resolution"}},
            {"match": {"metadata.session_year": 1672}},
            {"match": {"metadata.proposition_type": "requeste"}}
        ]
    }
}
aggs = {
    "months": {
        "date_histogram": {
            "field": "metadata.session_date",
            "calendar_interval": "month"
        }
    }
}
response = rep_es.es_anno.search(index="resolutions", query=query, aggs=aggs, size=0)
buckets = response["aggregations"]["months"]["buckets"]
for bucket in buckets:
    print(bucket["key_as_string"].split("T")[0], bucket["doc_count"])



1672-01-01 12
1672-02-01 8
1672-03-01 35
1672-04-01 17
1672-05-01 47
1672-06-01 13
1672-07-01 20
1672-08-01 19
1672-09-01 22
1672-10-01 21
1672-11-01 23
1672-12-01 25


In [7]:
response["hits"]["total"]

{'value': 262, 'relation': 'eq'}

In [12]:
query = {
    "bool": {
        "must": [
            #{"match": {"type": "resolution"}},
            {"match": {"metadata.session_year": 1672}}
        ]
    }
}
aggs = {
    "prop_types": {
        "terms": {
            "field": "metadata.proposition_type.keyword"
        }
    }
}
response = rep_es.es_anno.search(index="resolutions", query=query, aggs=aggs, size=0)
response

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2605, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'prop_types': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'missive', 'doc_count': 1540},
    {'key': 'requeste', 'doc_count': 262},
    {'key': 'memorie', 'doc_count': 32},
    {'key': 'declaratie', 'doc_count': 7},
    {'key': 'rapport', 'doc_count': 1},
    {'key': 'rekening', 'doc_count': 1}]}}}

In [11]:
from collections import Counter

tijden_counter = Counter()

for sent in res_sents:
    if 'tijden' in sent:
        index = sent.index('tijden')
        tijden_counter.update([sent[index-1]])
    if 'tyden' in sent:
        index = sent.index('tyden')
        tijden_counter.update([sent[index-1]])

for tijden, count in tijden_counter.most_common(20):
    print(tijden, count)

parsing file ../../data/paragraphs/resolutions-paragraphs-3285-3286.tsv.gz
van 88
in 14
allen 12
becommerlicke 4
voorige 4
perplexe 3
voorgaende 3
periculeuse 3
trouble 3
pressante 2
dangereuse 2
ongevallige 2
bij 2
dese 2
beswaerlijcke 2
immemoriale 1
vreedige 1
anderen 1
gesette 1
ter 1


In [14]:
para_file = "../../data/paragraphs/resolutions-paragraphs-3760-3769.tsv"

res_sents = text_helper.ResolutionSentences(para_file)

tijden_counter = Counter()

for sent in res_sents:
    if 'tijden' in sent:
        index = sent.index('tijden')
        tijden_counter.update([sent[index-1]])
    if 'tyden' in sent:
        index = sent.index('tyden')
        tijden_counter.update([sent[index-1]])

for tijden, count in tijden_counter.most_common(20):
    print(tijden, count)

parsing file ../../data/paragraphs/resolutions-paragraphs-3760-3769.tsv
van 106
allen 85
oude 72
in 36
voorige 29
alle 24
verscheyde 22
immemoriale 21
dese 11
der 8
voorgaende 7
andere 6
tegenwoordige 5
bedroefde 4
die 4
bekommerlijcke 4
ende 4
by 3
oorloghs 3
lange 3
